### --- Day 19: Not Enough Minerals ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/19

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 19);
WriteLines(inputLines);

Loading puzzle file: Day19.txt
Total lines: 30
Max line length: 161

Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 12 clay. Each geode robot costs 3 ore and 8 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 2 ore. Each obsidian robot costs 2 ore and 15 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 3: Each ore robot costs 4 ore. Each clay robot costs 3 ore. Each obsidian robot costs 4 ore and 18 clay. Each geode robot costs 4 ore and 11 obsidian.
Blueprint 4: Each ore robot costs 2 ore. Each clay robot costs 2 ore. Each obsidian robot costs 2 ore and 10 clay. Each geode robot costs 2 ore and 11 obsidian.
Blueprint 5: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 ore and 9 clay. Each geode robot costs 2 ore and 9 obsidian.


In [4]:
string[] testInputLines = [
    "Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.",
    "Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.",
];

Ok, so for this one, my first reaction is that this is another state-space exploration. We can model the states as a search - it looks like there is a tradeoff to be made at some point to stop building ore and clay robots so we can build obsidian robots, and subsequently geode cracking robots.

In [5]:
using Blueprint = (int id, int oreCost, int clayCost, int obsidianOreCost, int obsidianClayCost, int geodeOreCost, int geodeObsidianCost);

record MineState(int minutesRemain, int oreBots, int oreCount, int clayBots, int clayCount, int obsidBots, int obsidCount, int geodeBots, int geodeCount);

enum RobotChoice
{
    BuyOreRobot,
    BuyClayRobot,
    BuyObsidianRobot,
    BuyGeodeRobot,
    Nothing,
};

So this one turned out to be the first puzzle to give us some real showstopping challenges, in particular how to reduce the computation time given such a massive search space. There's a few intuitive tricks that immediately jump out, for instance, in the second-last minute, the only viable option is to build a geode robot if possible, since anything else will not contribute an additional geode in the last minute. But unfortunately we can't continue this working-backwawrds approach like a typical dynamic programming puzzle, as the exact number of resources are not known. 

Another intuition is that it makes no sense to just build ore and clay robots the whole time- you need to be making progress towards the goal. So we need some way to measure our progress and we can use that to prune unproductive branches. The first approach I tried is to compare states - that is, if one state has $m$ ore, $n$ ore robots, etc, then another state is considered worse if it has $<= m$ ore, $<= n$ robots, etc, for all resources and robots. I tried using this to prune branches that achieved lesser states, but it didn't yield the performance I was looking for.

Eventually, I asked an LLM to give me some hints. The key hint that unlocked a solution for me was to use a _theoretical best_ geode count as a progress measure. That is, assuming we could build a geode robot every minute for every remaining minute, how many geodes could we theoertically obtain? We can compare this theoretical geode count agains our best _actual_ geode count, and discard any paths that cannot theoretically improve upon our best. At this point I also switched to a depth-first search of the problem space, to help find an initial solution as quickly as possible and allow the pruning to take effect.

In [6]:
Blueprint ParseBlueprint(string inputLine)
{
    var ints = inputLine.ParseInts().ToArray();
    return (
        ints[0],
        ints[1],
        ints[2],    
        ints[3],
        ints[4],
        ints[5],
        ints[6]
    );
}

void TestAllBlueprints(string[] inputLines)
{
    var blueprints = inputLines.Select(ParseBlueprint).ToArray();

    foreach (var blueprint in blueprints)
    {
        // Console.WriteLine("Blueprint is...");
        // Console.WriteLine(blueprint);
        var best = RunOne(blueprint);
        Console.WriteLine($"Blueprint {blueprint.id} best is {best}");
    }
}

int RunOne(Blueprint blueprint, int minutesRemain = 24)
{
    MineState initial = new(minutesRemain, 1, 0, 0, 0, 0, 0, 0, 0);

    int BestPossible(MineState mineState)
    {
        var remain = mineState.minutesRemain;
        var existing = mineState.geodeBots * remain;

        // The absolute theoretical best we could do is to add geode robots
        // every minute until the end, giving us 1 + 2 + 3 + ... n-1 additional
        // geodes for n remaining minutes
        var theoretical = (remain - 1) * remain / 2;

        return mineState.geodeCount + existing + theoretical;
    }

    int best = 0;
    IEnumerable<MineState> GetNextStates(MineState initial)
    {
        if (initial.minutesRemain is 0)
        {
            yield break;
        }

        var bestPossible = BestPossible(initial);
        if (bestPossible <= best)
        {
            yield break;
        }

        var newOre = initial.oreBots;
        var newClay = initial.clayBots;
        var newObsid = initial.obsidBots;
        var newGeode = initial.geodeBots;
        
        foreach (var opt in Enum.GetValues<RobotChoice>())
        {
            var next = initial;

            next = opt switch {
                RobotChoice.BuyOreRobot => MakeOreBot(next),
                RobotChoice.BuyClayRobot => MakeClayBot(next),
                RobotChoice.BuyObsidianRobot => MakeObsidianBot(next),
                RobotChoice.BuyGeodeRobot => MakeGeodeRobot(next),
                _ => next
            };

            next = next with {
                minutesRemain = next.minutesRemain - 1,
                oreCount = next.oreCount + newOre,
                clayCount = next.clayCount + newClay,
                obsidCount = next.obsidCount + newObsid,
                geodeCount = next.geodeCount + newGeode
            };

            yield return next;
        }
    }

    var allStates = DFS(initial, GetNextStates);

    foreach (var state in allStates)
    {
        if (state.geodeCount > best)
        {
            best = state.geodeCount;
        }
    }
    
    return best;

    MineState MakeOreBot(MineState current) 
    {
        return blueprint.oreCost > current.oreCount
            ? current
            : current with {
                oreCount = current.oreCount - blueprint.oreCost,
                oreBots = current.oreBots + 1
            };
    }

    MineState MakeClayBot(MineState current)
    {
        return blueprint.clayCost > current.oreCount
            ? current
            : current with {
                oreCount = current.oreCount - blueprint.clayCost,
                clayBots = current.clayBots + 1
            };
    }

    MineState MakeObsidianBot(MineState current)
    {
        if (blueprint.obsidianClayCost > current.clayCount) { return current; }
        if (blueprint.obsidianOreCost > current.oreCount) { return current; }

        return current with {
            oreCount = current.oreCount - blueprint.obsidianOreCost,
            clayCount = current.clayCount - blueprint.obsidianClayCost,
            obsidBots = current.obsidBots + 1
        };
    }

    MineState MakeGeodeRobot(MineState current)
    {
        if (blueprint.geodeObsidianCost > current.obsidCount) { return current; }
        if (blueprint.geodeOreCost > current.oreCount) { return current; }

        return current with {
            obsidCount = current.obsidCount - blueprint.geodeObsidianCost,
            oreCount = current.oreCount - blueprint.geodeOreCost,

            geodeBots = current.geodeBots + 1
        };
    }
}

// Using blueprint 1 in the example above, the largest number of geodes you
// could open in 24 minutes is 9...

// However, by using blueprint 2 in the example above, you could do even better:
// the largest number of geodes you could open in 24 minutes is 12.

TestAllBlueprints(testInputLines);

Blueprint 1 best is 9
Blueprint 2 best is 12


In [7]:
int SumQualityLevels(string[] inputLines) 
    => inputLines.Select(ParseBlueprint)
        .Select(b => b.id * RunOne(b))
        .Sum();

In [8]:
// Finally, if you add up the quality levels of all of the blueprints in the
// list, you get 33.

var testAnswer = SumQualityLevels(testInputLines);
Console.WriteLine(testAnswer);

33


In [9]:
// Determine the quality level of each blueprint using the largest number of
// geodes it could produce in 24 minutes. What do you get if you add up the
// quality level of all of the blueprints in your list?

var part1Answer = SumQualityLevels(inputLines);
Console.WriteLine(part1Answer);

1306


In [10]:
// 1306 is correct!
Ensure(1306, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/19

Ok, so for part 2, we only need to consider the first three blueprints, but we need to search to a depth of 32. That's not a huuuge difference from part 1, so hopefully our pruning strategy from part 1 is enough to get us through here. Given the time spent on part 1, I'd be happy to just get the result here and move on :)

In [12]:
void TestAllBlueprints2(string[] inputLines)
{
    var blueprints = inputLines.Select(ParseBlueprint).ToArray();

    foreach (var blueprint in blueprints.Take(3))
    {
        // Console.WriteLine("Blueprint is...");
        // Console.WriteLine(blueprint);
        var best = RunOne(blueprint, minutesRemain: 32);
        Console.WriteLine($"Blueprint {blueprint.id} best is {best}");
    }
}

// In 32 minutes, the largest number of geodes blueprint 1 (from the example
// above) can open is 56...

// However, blueprint 2 from the example above is still better; using it, the
// largest number of geodes you could open in 32 minutes is 62.

TestAllBlueprints2(testInputLines);

Blueprint 1 best is 56
Blueprint 2 best is 62


In [13]:
// You no longer have enough blueprints to worry about quality levels. Instead,
// for each of the first three blueprints, determine the largest number of
// geodes you could open; then, multiply these three values together.

int MultiplyBestGeodes(string[] inputLines)
    => inputLines.Select(ParseBlueprint).Take(3)
    .Select(b => RunOne(b, minutesRemain: 32))
    .Aggregate(1, (a, b) => a * b);

In [14]:
// Don't worry about quality levels; instead, just determine the largest number
// of geodes you could open using each of the first three blueprints. What do
// you get if you multiply these numbers together?

var part2Answer = MultiplyBestGeodes(inputLines);
Console.WriteLine(part2Answer);

37604


In [15]:
// 37604 is correct!
Ensure(37604, part2Answer);